# Experiment 4: SANLI pipeline
***

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from datasets import Dataset

from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import transformers

import torch

from ailignment.datasets.util import get_accuracy_metric
from ailignment.datasets.moral_stories import make_action_classification_dataframe, get_random_value_dataset
import ailignment.datasets.moral_stories_clustered as msc
from ailignment.training import sequence_classification

pd.set_option('display.max_colwidth', 400)
dataframe = pd.read_pickle("../data/moral_stories_proto_l2s.dat")
# a function to convert train and test dataframes to huggingface datasets
def convert_hf_dataset(tokenizer, train, padding="max_length"):
    def tok(samples):
        return tokenizer(samples["action"], samples["norm_storyfied"], padding=padding, 
                         truncation=True, return_token_type_ids=True)

    train_data = Dataset.from_pandas(train)
    train_data = train_data.map(tok, batched=True)
    return train_data

def convert_hf_dataset_concat(tokenizer, train, padding="max_length"):
    def tok(samples):
        x = [a+ " " + b for a,b in zip(samples["action"], samples["norm_action"])]
        return tokenizer(x, padding=padding, 
                         truncation=True, return_token_type_ids=True)

    train_data = Dataset.from_pandas(train)
    train_data = train_data.map(tok, batched=True)
    return train_data

In [4]:
name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForSequenceClassification.from_pretrained(name)

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
training_args = TrainingArguments(
    output_dir="/data/kiehne/results/shuffled_values/trash/",
    per_device_eval_batch_size=32,
    no_cuda=True)
trainer = Trainer(
    model=model,
    args=training_args,
)

In [8]:
results = []
for p in tqdm(np.linspace(0,1,11)):
    # prepare dataset
    data = get_random_value_dataset(dataframe, p=p, top_n=20)
    train, test = make_action_classification_dataframe(data, test_size=0.1)
    dataset =  pd.concat([train, test], axis=0)
    dataset["sentiment"] = dataset["norm_sentiment"].apply(lambda x: int(x=="POSITIVE"))
    dataset_hf = convert_hf_dataset(tokenizer, dataset)
    # run entailment model
    r = trainer.predict(dataset_hf)
    scores = torch.softmax(torch.from_numpy(r.predictions), 1, torch.float32).numpy()
    # find entailed samples
    is_entailed = (scores[:,0] > scores[:,2]).astype("int32")
    labels = np.array(dataset["labels"])
    sentiment = np.array(dataset["sentiment"])
    # according to our theory, equality of sentiment and entailment implies moral behavior
    # and the opposite is true for contradictory and immorality.
    y_pred = (is_entailed == sentiment).astype("int32")
    acc = (y_pred == labels).mean()
    results.append(acc)
    print("Accuracy:", acc)
print(results)

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: norm_action, sentiment, consequence, action, actor_name, l2s_output, norm_sentiment, flipped, ID, __index_level_0__, intention, norm, norm_value, norm_storyfied, situation.
***** Running Prediction *****
  Num examples = 23992
  Batch size = 64
  9%|▉         | 1/11 [09:56<1:39:23, 596.39s/it]

Accuracy: 0.7954318106035345


  0%|          | 0/24 [00:00<?, ?ba/s]

...

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: norm_action, sentiment, consequence, action, actor_name, l2s_output, norm_sentiment, flipped, ID, __index_level_0__, intention, norm, norm_value, norm_storyfied, situation.
***** Running Prediction *****
  Num examples = 23992
  Batch size = 64
 91%|█████████ | 10/11 [1:35:55<08:42, 522.62s/it]

Accuracy: 0.7954318106035345


  0%|          | 0/24 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: norm_action, sentiment, consequence, action, actor_name, l2s_output, norm_sentiment, flipped, ID, __index_level_0__, intention, norm, norm_value, norm_storyfied, situation.
***** Running Prediction *****
  Num examples = 23992
  Batch size = 64
100%|██████████| 11/11 [1:44:24<00:00, 569.53s/it]

Accuracy: 0.7954318106035345
[0.7954318106035345, 0.7954318106035345, 0.7954318106035345, 0.7954318106035345, 0.7954318106035345, 0.7954318106035345, 0.7954318106035345, 0.7954318106035345, 0.7954318106035345, 0.7954318106035345, 0.7954318106035345]


In [9]:
results

[0.7954318106035345,
 0.7954318106035345,
 0.7954318106035345,
 0.7954318106035345,
 0.7954318106035345,
 0.7954318106035345,
 0.7954318106035345,
 0.7954318106035345,
 0.7954318106035345,
 0.7954318106035345,
 0.7954318106035345]

In [7]:
sample = dataframe.sample(100)
data = get_random_value_dataset(sample, p=0, top_n=20)
train, test = make_action_classification_dataframe(data, test_size=0.1)
dataset =  pd.concat([train, test], axis=0)
dataset["sentiment"] = dataset["norm_sentiment"].apply(lambda x: int(x=="POSITIVE"))

In [8]:
# take a look at misclassified samples
# prepare dataset
dataset_hf = convert_hf_dataset(tokenizer, dataset)
# run entailment model
r = trainer.predict(dataset_hf)
scores = torch.softmax(torch.from_numpy(r.predictions), 1, torch.float32).numpy()
# find entailed samples
is_entailed = (scores[:,0] > scores[:,2]).astype("int32")
labels = np.array(dataset["labels"])
sentiment = np.array(dataset["sentiment"])
# according to our theory, equality of sentiment and entailment implies moral behavior
# and the opposite is true for contradictory and immorality.
y_pred = (is_entailed == sentiment).astype("int32")
acc = (y_pred == labels).mean()
dataset["y_pred"] = y_pred
dataset["is_entailed"] = is_entailed
dataset["pred_correct"] = y_pred == labels
print("Accuracy:", acc)

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: norm_storyfied, flipped, situation, action, norm, norm_action, intention, l2s_output, consequence, sentiment, __index_level_0__, actor_name, ID, norm_sentiment, norm_value.
***** Running Prediction *****
  Num examples = 200
  Batch size = 32


Accuracy: 0.785


In [22]:
dataset[["action", "norm","norm_sentiment","norm_storyfied","flipped",
         "labels","sentiment","y_pred","is_entailed","pred_correct"]][dataset["labels"]==1].tail(20)

,action,norm,norm_sentiment,norm_storyfied,flipped,labels,sentiment,y_pred,is_entailed,pred_correct
131,John admits he ate it but tells Mindy he also bought her a brand new one as well.,should telling people the truth.,POSITIVE,John tells people the truth,False,1,1,0,0,False
156,Billy studies a dictionary before playing scrabble and wins.,It's hurtful cheating when playing a game.,NEGATIVE,Billy cheats when playing a game,False,1,0,1,0,True
128,"Kyle combs his hair, trims his beard, and puts on washed and ironed clothes.",It's fine making yourself look nice for people.,POSITIVE,Kyle makes himself look nice for people,False,1,1,1,1,True
162,Don uses a scented product which is offensive to dog noses on the furniture.,It's hurtful locking your pet up in closed quarters.,NEGATIVE,Don locks his pet up in closed quarters,False,1,0,1,0,True
116,"Paul buys shots for all of his friends except Wayne, because he needs to be sober to drive them home.",is wrong forcing someone to drink alcohol.,NEGATIVE,Paul forces someone to drink alcohol,False,1,0,0,1,False
95,Julie agrees to go on a second date with Michael and has a great experience.,It's kind going on a second date.,POSITIVE,Julie goes on a second date,False,1,1,1,1,True
171,Jessica wears a mask and gloves so that she can give her mom hugs.,It's morally wrong refusing to hug your mother.,NEGATIVE,Jessica refuses to hug her mother,False,1,0,1,0,True
123,Ron keeps panning for gold and finds a nice bit to take home.,shouldn't stealing someone's gold when they find it.,NEGATIVE,Ron steals someone's gold when they find it,False,1,0,1,0,True
119,Toby says that he'll have to ask his friend if they still want the movie before giving it away.,It's bad giving away another person's things without permission.,NEGATIVE,Toby gives away another person's things without permission,False,1,0,0,1,False
106,Sue stays inside and calls animal control about the situation.,should leaving injured animals to the professionals.,POSITIVE,Sue leaves injured animals to the professionals,False,1,1,1,1,True
